In [7]:
import pandas as pd
import numpy as np
import torch

import scGeneRAI
import os
import networkx as nx

In [ ]:
# ex_data = pd.read_csv('../scGeneRAI/data/example_data.csv').iloc[:,3:]
# ex_data_descriptors = pd.read_csv('../scGeneRAI/data/example_data.csv').iloc[:,1:3]

In [ ]:
raw = pd.read_csv('../scGeneRAI/data/raw_data.csv')
filtered_raw = raw[ raw['Cell.Type'].str.count(',') == 0]
filtered_raw = filtered_raw[filtered_raw['Cell.Type'] != 'Unknown']
ex_data = filtered_raw.iloc[:,13:]
ex_data_descriptors = filtered_raw.iloc[:,[2,12]]
data = pd.concat([ex_data_descriptors, ex_data], axis=1)
# column_names = data.columns.tolist()
# column_names[0]
df_sorted = data.sort_values(by=['Cell.Type', 'orig.ident'])
df_sorted.to_csv('./data/sorted_data.csv', index=False)

In [2]:
raw = pd.read_csv('../scGeneRAI/data/sorted_data.csv')
ex_data = raw.iloc[:,2:]
ex_data_descriptors = raw.iloc[:,:2]
ex_data_descriptors

,orig.ident,Cell.Type
0,Control,Columella
1,Control,Columella
2,Control,Columella
3,Control,Columella
4,Control,Columella
...,...,...
11527,DC3000,Xylem
11528,DC3000,Xylem
11529,DC3000,Xylem
11530,DC3000,Xylem


In [3]:
means = ex_data.mean(axis=0)
sds = ex_data.std(axis=0)

ex_data = (ex_data-means)/sds

In [ ]:
ex_data

In [ ]:
ex_data_descriptors.iloc[:61,:]

In [ ]:
len(ex_data_descriptors)

In [ ]:
ex_data_descriptors.iloc[61:984,:]

In [13]:
model = scGeneRAI.scGeneRAI(ex_data, model_depth =2, descriptors = ex_data_descriptors)

In [9]:
model.fit(ex_data, nepochs = 5, model_depth =2, descriptors = ex_data_descriptors, early_stopping=False, device_name = torch.device('cuda'))


100%|██████████| 5/5 [01:14<00:00, 14.95s/it]

the network trained for 5 epochs (testloss: 1.0691689252853394)


In [12]:
torch.save(model.nn.state_dict(), 'saved_model.pt')

In [14]:
model.nn.load_state_dict(torch.load('saved_model.pt'))

<All keys matched successfully>

In [16]:
preds = model.predict_networks(ex_data.iloc[:61,:], descriptors = ex_data_descriptors.iloc[:61,:], PATH = '.', device_name = torch.device('cuda'))

0


KeyboardInterrupt: 

In [ ]:
preds1

In [ ]:
network_data = pd.concat(preds)

In [ ]:
network_data

In [ ]:
files = os.listdir('./results')

In [ ]:
network_data = pd.concat([pd.read_csv('./results/' + file) for file in files])

In [ ]:
network_data['LRP'] = np.abs(network_data['LRP'])
network_data = network_data[network_data['source_gene'] != network_data['target_gene']]

average_network = network_data[['LRP', 'source_gene', 'target_gene']].groupby(['source_gene', 'target_gene']).mean().reset_index()

In [ ]:
edges = preds.sort_values(by='LRP', ascending=False)#.iloc[:200,:]
edges

In [ ]:
network = nx.from_pandas_edgelist(edges, source='source_gene', target='target_gene', edge_attr='LRP')

In [ ]:
nx.draw(network, with_labels=False, node_size=0, node_color='white', width = edges['LRP']*0.1)

In [ ]:
preds = model.predict_networks(ex_data.iloc[61:984,:], descriptors = ex_data_descriptors.iloc[61:984,:], PATH = '.', device_name = torch.device('cuda'))

In [ ]:
network_data = pd.concat([preds1, preds])

In [ ]:
final_network = network_data[['LRP', 'source_gene', 'target_gene']].groupby(['source_gene', 'target_gene']).sum().reset_index()

In [ ]:
final_network['LRP'] = final_network['LRP'] / 923
final_network

In [ ]:
edges = final_network.sort_values(by='LRP', ascending=False)#.iloc[:200,:]
edges

In [ ]:
network = nx.from_pandas_edgelist(edges, source='source_gene', target='target_gene', edge_attr='LRP')

In [ ]:
nx.draw(network, with_labels=False, node_size=0, node_color='white', width = edges['LRP']*0.1)

In [ ]:
len(ex_data_descriptors.iloc[984:1148,:])

In [ ]:
model = scGeneRAI.scGeneRAI(ex_data, model_depth =2, descriptors = ex_data_descriptors)

In [ ]:
model.nn = torch.load('saved_model.pt')

In [ ]:
preds1 = model.predict_networks(ex_data.iloc[:61,:], descriptors = ex_data_descriptors.iloc[:61,:], PATH = '.', device_name = torch.device('cuda'))